In [1]:
# Put these at the top of every notebook, to get automatic reloading and inline plotting
%reload_ext autoreload
%autoreload 2
%matplotlib inline

In [2]:
# This file contains all the main external libs we'll use
from fastai.imports import *

/home/paperspace/anaconda3/envs/fastai/lib/python3.6/site-packages/sklearn/ensemble/weight_boosting.py:29: DeprecationWarning: numpy.core.umath_tests is an internal NumPy module and should not be imported. It will be removed in a future NumPy release.
  from numpy.core.umath_tests import inner1d


In [3]:
from fastai.transforms import *
from fastai.conv_learner import *
from fastai.model import *
from fastai.dataset import *
from fastai.sgdr import *
from fastai.plots import *

In [4]:
PATH = "data/force/"
sz=224
arch=resnet50
bs=32


In [5]:
os.listdir(f'{PATH}')

['valid_20', 'test', 'models', 'train', 'tmp', 'train_80']

In [ ]:
os.listdir(f'{PATH}test')

In [ ]:
files = os.listdir(f'{PATH}train/1')[:5]
files

In [ ]:
img = plt.imread(f'{PATH}train/1/{files[0]}')
plt.imshow(img);

In [ ]:

img.shape

In [ ]:
tfms = tfms_from_model(arch, sz, aug_tfms=transforms_top_down, max_zoom=1.1)

In [ ]:
def split_dataset_into_test_and_train_sets(all_data_dir, training_data_dir, testing_data_dir, testing_data_pct):
    # Recreate testing and training directories
    if testing_data_dir.count('/') > 1:
        shutil.rmtree(testing_data_dir, ignore_errors=False)
        os.makedirs(testing_data_dir)
        print("Successfully cleaned directory " + testing_data_dir)
    else:
        print("Refusing to delete testing data directory " + testing_data_dir + " as we prevent you from doing stupid things!")

    if training_data_dir.count('/') > 1:
        shutil.rmtree(training_data_dir, ignore_errors=False)
        os.makedirs(training_data_dir)
        print("Successfully cleaned directory " + training_data_dir)
    else:
        print("Refusing to delete testing data directory " + training_data_dir + " as we prevent you from doing stupid things!")

    num_training_files = 0
    num_testing_files = 0

    for subdir, dirs, files in os.walk(all_data_dir):
        category_name = os.path.basename(subdir)

        # Don't create a subdirectory for the root directory
        print(category_name + " vs " + os.path.basename(all_data_dir))
        if category_name == os.path.basename(all_data_dir):
            continue

        training_data_category_dir = training_data_dir + '/' + category_name
        testing_data_category_dir = testing_data_dir + '/' + category_name

        if not os.path.exists(training_data_category_dir):
            os.mkdir(training_data_category_dir)

        if not os.path.exists(testing_data_category_dir):
            os.mkdir(testing_data_category_dir)

        for file in files:
            input_file = os.path.join(subdir, file)
            if np.random.rand(1) < testing_data_pct:
                shutil.copy(input_file, testing_data_dir + '/' + category_name + '/' + file)
                num_testing_files += 1
            else:
                shutil.copy(input_file, training_data_dir + '/' + category_name + '/' + file)
                num_training_files += 1

    print("Processed " + str(num_training_files) + " training files.")
    print("Processed " + str(num_testing_files) + " testing files.")

In [ ]:
train_fn=(f'{PATH}train_80')
valid_fn=(f'{PATH}valid_20')
test_fn=(f'{PATH}test')

os.makedirs(train_fn);os.makedirs(valid_fn)

In [ ]:

split_dataset_into_test_and_train_sets(f'{PATH}train', train_fn, valid_fn, 0.20)


In [ ]:
os.listdir(f'{train_fn}') == os.listdir(f'{valid_fn}')

In [ ]:
PATH

In [ ]:
data = ImageClassifierData.from_paths(PATH, bs=32, tfms=tfms, trn_name='train_80', val_name='valid_20', test_name='test', num_workers=8)




In [ ]:
size_d = {k: PIL.Image.open(PATH+k).size for k in data.trn_ds.fnames}
print (size_d)

In [ ]:

row_sz, col_sz = list(zip(*size_d.values()))

In [ ]:
row_sz=np.array(row_sz); col_sz=np.array(col_sz)


In [ ]:
plt.hist(row_sz[row_sz<1000])

In [ ]:
plt.hist(col_sz[col_sz<1000])

In [ ]:
len(data.trn_ds), len(data.test_ds)


In [ ]:
len(data.classes), data.classes[:5]

In [ ]:
def get_data(sz, bs, train, valid, test):
    tfms = tfms_from_model(arch, sz, aug_tfms=transforms_top_down, max_zoom=1.1)
    data = ImageClassifierData.from_paths(PATH, bs=32, tfms=tfms, trn_name=train, 
                                          val_name=valid, test_name=test, num_workers=6)

    return data if sz>400 else data.resize(440, 'tmp')

In [ ]:
train='train_95'
valid= 'valid_5'
test='test'
sz=300
bs=32
data = get_data(sz, bs, train, valid, test)

In [ ]:
learn = ConvLearner.pretrained(arch, data, precompute=True)

In [ ]:
??resnet

In [ ]:
??get_fc_layers

In [ ]:
lrf=learn.lr_find()

In [ ]:
learn.sched.plot_lr()

In [ ]:
learn.sched.plot()

In [ ]:
learn.fit(1e-2, 10)

In [ ]:
learn.precompute=False

In [ ]:
learn.fit(1e-2, 20, cycle_len=1)

In [ ]:
learn.save('399_lastlayer')

In [ ]:
learn.load('399_lastlayer')

In [ ]:
learn.unfreeze()

In [ ]:
lr=1e-2
lr=np.array([lr/9,lr/3,lr])

In [ ]:
learn.fit(lr,10, cycle_len=1)

In [ ]:
learn.sched.plot_lr()

In [ ]:
learn.save('399_all_22feb_v2')

In [ ]:
learn.load('399_all_22feb')
    

In [ ]:
log_preds,y = learn.TTA()
probs = np.mean(np.exp(log_preds),0)
accuracy_np(probs, y)

In [ ]:
accuracy_np(probs, y)

In [ ]:
from sklearn.metrics import precision_recall_fscore_support


In [ ]:
y_preds = np.argmax(probs, axis=1)
precision_recall_fscore_support(y, y_preds, average='micro')

In [ ]:
from sklearn.metrics import confusion_matrix
cm = confusion_matrix(y, y_preds)
plot_confusion_matrix(cm, data.classes)

In [ ]:
## no do it for the test set

In [ ]:
log_preds, y= learn.TTA(is_test=True)
probs = np.mean(np.exp(log_preds),0)
# no accurarcy as we do not know what the test set is

In [ ]:
data.classes[np.argmax(probs[0])]

test_label=[]; test_class=[]
for index, p in enumerate(probs[:]):
    test_label.append(data.test_ds.fnames[index][5:]) 
    test_class.append(data.classes[np.argmax(p)])


In [ ]:
d = {'file': test_label, 'species': test_class}
df = pd.DataFrame(data=d)


In [ ]:
df.head()

In [ ]:
SUBM = f'{PATH}subm/'; print(SUBM)
os.makedirs(SUBM, exist_ok=True)
df.to_csv(f'{SUBM}subm.csv', index=False)

In [ ]:
FileLink(f'{SUBM}subm.csv')